In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
curr_app = pd.read_csv('application_train.csv')
y = curr_app['TARGET']
X = curr_app.iloc[:, :1]

In [12]:
X_test = pd.read_csv('application_test.csv')
X_test = X_test.iloc[:, :1]

In [13]:
pos_df = pd.read_csv('POS_CASH_balance.csv')

In [14]:
pos_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [15]:
pos_df.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [16]:
pos_df = pd.get_dummies(pos_df)

In [17]:
gb_sum_df = pos_df.iloc[:, 1:].groupby('SK_ID_CURR').sum().reset_index()

In [18]:
gb_sum_df['COUNT'] = pos_df[['SK_ID_CURR']].groupby('SK_ID_CURR').size().reset_index().iloc[:, 1]

In [19]:
gb_sum_df['CNT_INSTALMENT_MAX'] = pos_df[['CNT_INSTALMENT', 'SK_ID_CURR']].groupby('SK_ID_CURR').max().reset_index().iloc[:, 1]
gb_sum_df['CNT_INSTALMENT_MIN'] = pos_df[['CNT_INSTALMENT', 'SK_ID_CURR']].groupby('SK_ID_CURR').min().reset_index().iloc[:, 1]
gb_sum_df['CNT_INSTALMENT_AVG'] = pos_df[['CNT_INSTALMENT', 'SK_ID_CURR']].groupby('SK_ID_CURR').mean().reset_index().iloc[:, 1]

In [20]:
gb_sum_df['INSTALMENTS_REMAINING'] = (pos_df[['SK_ID_CURR','SK_ID_PREV','CNT_INSTALMENT_FUTURE']]
                                      .groupby(['SK_ID_CURR','SK_ID_PREV']).min()
                                      .groupby('SK_ID_CURR').sum().reset_index().iloc[:,1])

In [21]:
gb_sum_df['MONTHS_BALANCE_AVG'] = (pos_df[['SK_ID_CURR','MONTHS_BALANCE']]
 .groupby(['SK_ID_CURR']).mean().reset_index().iloc[:,1]
)

gb_sum_df['MONTHS_BALANCE_MAX'] = (pos_df[['SK_ID_CURR','MONTHS_BALANCE']]
 .groupby(['SK_ID_CURR']).max().reset_index().iloc[:,1]
)
gb_sum_df['MONTHS_BALANCE_MIN'] = (pos_df[['SK_ID_CURR','MONTHS_BALANCE']]
 .groupby(['SK_ID_CURR']).min().reset_index().iloc[:,1]
)

In [22]:
gb_sum_df['SK_DPD_AVG'] = (pos_df[['SK_ID_CURR','SK_DPD']]
 .groupby(['SK_ID_CURR']).mean().reset_index().iloc[:,1]
)

gb_sum_df['SK_DPD_DEF_AVG'] = (pos_df[['SK_ID_CURR','SK_DPD_DEF']]
 .groupby(['SK_ID_CURR']).mean().reset_index().iloc[:,1]
)

In [23]:
gb_sum_df['CNT_INSTALMENT_NA'] = pd.concat([pos_df[['CNT_INSTALMENT']].isnull(), pos_df[['SK_ID_CURR']]], axis=1).groupby('SK_ID_CURR').sum().reset_index().iloc[:, 1]
gb_sum_df['CNT_INSTALMENT_FUTURE_NA'] = pd.concat([pos_df[['CNT_INSTALMENT_FUTURE']].isnull(), pos_df[['SK_ID_CURR']]], axis=1).groupby('SK_ID_CURR').sum().reset_index().iloc[:, 1]

In [24]:
X = X.merge(gb_sum_df, on='SK_ID_CURR', how='left')
X_test = X_test.merge(gb_sum_df, on='SK_ID_CURR', how='left')

In [25]:
corrs = pd.Series([y.corr(X[col]) for col in X.columns[-gb_sum_df.shape[1]+1:]], index=gb_sum_df.columns[1:]).sort_values()

In [26]:
corrs = abs(corrs).sort_values(ascending=False)
corrs

MONTHS_BALANCE_MIN                            0.055307
MONTHS_BALANCE                                0.040570
NAME_CONTRACT_STATUS_Active                   0.035866
COUNT                                         0.035632
MONTHS_BALANCE_AVG                            0.034543
CNT_INSTALMENT_MIN                            0.019840
NAME_CONTRACT_STATUS_Completed                0.019547
CNT_INSTALMENT_AVG                            0.018066
CNT_INSTALMENT                                0.014670
CNT_INSTALMENT_MAX                            0.013296
CNT_INSTALMENT_FUTURE_NA                      0.010643
CNT_INSTALMENT_NA                             0.010374
INSTALMENTS_REMAINING                         0.009569
NAME_CONTRACT_STATUS_Returned to the store    0.009103
SK_DPD_DEF_AVG                                0.006496
CNT_INSTALMENT_FUTURE                         0.005881
NAME_CONTRACT_STATUS_Amortized debt           0.005526
SK_DPD_AVG                                    0.005436
SK_DPD_DEF

In [27]:
X[corrs.index].to_pickle('features_pos.pkl')
X_test[corrs.index].to_pickle('features_pos_test.pkl')